1. Interpretación y Ejecución de Comandos
El siguiente código muestra cómo inicializar el Kernel y ejecutar una función basada en la entrada del usuario.

In [ ]:
import os
from azure.cosmos import CosmosClient, exceptions
from azure.identity import DefaultAzureCredential



ENVIRONMENT = os.getenv("ENVIRONMENT", "development")

if ENVIRONMENT == "development":
    DEFAULT_CREDENTIAL = DefaultAzureCredential()
else:
    DEFAULT_CREDENTIAL = os.getenv("COSMOS_KEY", "")


COSMOS_ENDPOINT = os.getenv("COSMOS_ENDPOINT", "")
COSMOS_KEY = os.getenv("COSMOS_KEY", "")
DATABASE_NAME = os.getenv("MANAGER_DATABASE", "CommandContainer")


class CommandDatabase:

    def __init__(self, container_name):
        self.client = CosmosClient(COSMOS_ENDPOINT, DEFAULT_CREDENTIAL)  #type: ignore
        self.database = self.client.get_database_client(DATABASE_NAME)
        self.container = self.database.get_container_client(container_name)

    def create_command(self, command):
        try:
            self.container.create_item(body=command)
            return "Command created successfully"
        except exceptions.CosmosHttpResponseError as e:
            return f"An error occurred: {e.message}"

    def read_command(self, command_id):
        try:
            command = self.container.read_item(item=command_id, partition_key={})
            return command
        except exceptions.CosmosResourceNotFoundError:
            return "Command not found"
        except exceptions.CosmosHttpResponseError as e:
            return f"An error occurred: {e.message}"

    def list_command(self):
        try:
            command = list(self.container.read_all_items())
            return command
        except exceptions.CosmosResourceNotFoundError:
            return "Command not found"
        except exceptions.CosmosHttpResponseError as e:
            return f"An error occurred: {e.message}"

    def update_command(self, updated_command):
        try:
            self.container.upsert_item(body=updated_command)
            return "Command updated successfully"
        except exceptions.CosmosResourceNotFoundError:
            return "Command not found"
        except exceptions.CosmosHttpResponseError as e:
            return f"An error occurred: {e.message}"

    def delete_command(self, command_id):
        try:
            self.container.delete_item(item=command_id, partition_key={})
            return "Command deleted successfully"
        except exceptions.CosmosResourceNotFoundError:
            return "Command not found"
        except exceptions.CosmosHttpResponseError as e:
            return f"An error occurred: {e.message}"


In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.functions import kernel_function, KernelArguments
from semantic_kernel.connectors.ai.azure_ai_inference import AzureAIInferenceChatCompletion

class SaludosSkill:
    # Define your function as a plain method.
    # Its name (e.g. "saludar") will be used when invoking it.
    @kernel_function(name="saludar")
    async def saludar(self, nombre: str) -> str:
        return f"Hola, {nombre}!"
    

class OperacaoBancoDados:

    def __init__(self, auth: str):
        self.database = {}
        self.auth = auth or ""
    # Define your function as a plain method.
    # Its name (e.g. "saludar") will be used when invoking it.
    @kernel_function(name="retrieve")
    async def saludar(self, nombre: str) -> str:
        return database.retrieve(nombre)

    @kernel_function(name="write")
    async def saludar(self, nombre: str) -> str:
        return database.write(nombre)    


async def main():
    # Create a new kernel instance directly.
    kernel = Kernel()

    # Import (register) your skill under a plugin name, e.g. "local".
    # The add_plugin method registers public methods of the given class.
    kernel.add_plugin("local", CommandDatabase("auth"))

    # Define the input.
    entrada_usuario = "Juan"

    # Invoke the function.
    # In this version, you reference the function as "local.saludar" (plugin_name.function_name)
    resultado = await kernel.run("local.read_command", id=entrada_usuario)
    print(resultado)

await main()

TypeError: plugin_name must be a string.

2. Orquestación de Flujos de Trabajo
Este código muestra cómo encadenar múltiples funciones en un flujo de trabajo.

In [4]:
@kernel_function(name="obtener_nombre")
def obtener_nombre() -> str:
    return "Juan"

@kernel_function(name="generar_mensaje")
def generar_mensaje(nombre: str) -> str:
    return f"Hola, {nombre}, ¿cómo estás?"


class CalableClass:

    async def __call__(self):
        return "Juan"


calable_class_function = CalableClass()

# Registrar funciones
kernel.add_function("obtener_nombre", obtener_nombre)
kernel.add_function("generar_mensaje", generar_mensaje)
kernel.add_function("calable_class_function", calable_class_function)

args = KernelArguments(nombre=entrada_usuario)

# Orquestación del flujo
nombre = await kernel.invoke(
    function_name="obtener_nombre", plugin_name="obtener_nombre"
)
mensaje = await kernel.invoke(
    function_name="generar_mensaje", plugin_name="generar_mensaje", arguments=args
)
print(mensaje)  # Salida: Hola, Juan, ¿cómo estás?


NameError: name 'kernel' is not defined

3. Gestión de Memoria y Contexto

Este código demuestra cómo almacenar y recuperar información del contexto.
Este codigo no se pasará porque no tenemos el storage factual, pero esta es la logica de storage

In [ ]:
from semantic_kernel.memory import SemanticTextMemory

# Crear memoria en el Kernel
memoria = SemanticTextMemory()

# Almacenar contexto
memoria.save_information("usuario_nombre", "Juan")

# Recuperar contexto y generar una respuesta
nombre = memoria.get("usuario_nombre")
print(f"Bienvenido de nuevo, {nombre}")  # Salida: Bienvenido de nuevo, Juan


4. Optimización de Planificación
Uso de un Planner para optimizar la ejecución de tareas.

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.planners import SequentialPlanner, FunctionCallingStepwisePlanner

# Configura tu servicio AI (reemplaza "YOUR_API_KEY" por tu API key y ajusta el modelo si es necesario)
chat_completion_service_4o = AzureChatCompletion(
    deployment_name="my-deployment",  
    api_key="my-api-key",
    endpoint="my-api-endpoint", # Used to point to your service
    service_id="openai_gpt4o", # Optional; for targeting specific services within Semantic Kernel
)

chat_completion_service_4o_mini = AzureChatCompletion(
    deployment_name="my-deployment",  
    api_key="my-api-key",
    endpoint="my-api-endpoint", # Used to point to your service
    service_id="openai_gpt4o_mini", # Optional; for targeting specific services within Semantic Kernel
)

chat_completion_service_phi4 = AzureChatCompletion(
    deployment_name="my-deployment",  
    api_key="my-api-key",
    endpoint="my-api-endpoint", # Used to point to your service
    service_id="phi_4", # Optional; for targeting specific services within Semantic Kernel
)


# Crea la instancia del kernel y registra el servicio AI con un ID, por ejemplo "openai"
kernel = Kernel(services={
    "gpt4o": chat_completion_service_4o,
    "gpt4omini": chat_completion_service_4o_mini,
    "phi4": chat_completion_service_phi4,
    },
)

# Usa ese servicio registrado en el planner
planner = SequentialPlanner(kernel, service_id="gpt4o")

# Crea el plan (await the creation to obtain the plan object)
plan = await planner.create_plan([
    {"function": "obtener_nombre"},
    {"function": "generar_mensaje", "args": {"nombre": "$0"}}
])

# Invoca el plan ya construido con kernel.invoke
primer_resultado = await kernel.invoke(function=plan)

planner_dos = SequentialPlanner(kernel, service_id="phi_4")
plan_dos = await planner.create_plan([
    {"function": "generar_mensaje", "args": {"nombre": primer_resultado}},
    {"function": "saludar", "args": {"nombre": "$0"}}
])

resultado = await kernel.invoke(function=plan_dos)

print(resultado)  # Debe imprimir: Hola, Juan, ¿cómo estás?